In [1]:
import calendar
import pandas as pd
import numpy as np
import panel as pn
from datetime import date, timedelta
from sqlalchemy import create_engine
from itables import show
pn.extension('tabulator')

engine = create_engine("sqlite:///c:\\ruby\\port_lite\\db\\development.sqlite3")
conlite = engine.connect()

engine = create_engine("sqlite:///c:\\ruby\\portmy\\db\\development.sqlite3")
conmy = engine.connect()

engine = create_engine("mysql+pymysql://root:@localhost:3306/stock")
const = engine.connect()

data_path = "../data/"
csv_path = "\\Users\\User\\iCloudDrive\\"
box_path = "\\Users\\User\\Dropbox\\"
one_path = "\\Users\\User\\OneDrive\\Documents\\Data\\"

today = date.today()
today

datetime.date(2022, 8, 7)

### Tables in the process

In [2]:
cols = 'trade name qty price active reason market xdate'.split()
colt = 'trans name spd reason qty target current percent active xdate'.split()

format_dict = {
    'qty':'{:,}','available_qty':'{:,}',
    'price':'{:.2f}','cost':'{:.2f}','max_price':'{:.2f}','min_price':'{:.2f}',
    'buy_target':'{:.2f}','sell_target':'{:.2f}','target':'{:.2f}','current':'{:.2f}',
    'beta':'{:.2f}',
    'amount':'{:,.2f}','sell_amt':'{:,.2f}','volume':'{:,.2f}',
    'percent':'{:,.2f}',
    'shares':'{:,}',    
    'q4':'{:.4f}','q3':'{:.4f}','q2':'{:.4f}','q1':'{:.4f}','dividend':'{:.4f}',
    'xdate':'{:%Y-%m-%d}','paiddate':'{:%Y-%m-%d}',
}

In [3]:
sql = '''
SELECT *
FROM orders
ORDER BY id DESC
LIMIT 1'''
orders = pd.read_sql(sql, conlite)
orders

,id,trade,name,qty,price,active,reason,market,xdate
0,18,B,PTTEP,300,146,2,RD05pct,SET50,2022-02-02


In [4]:
names = orders['name']
name = names.to_string(index=False)

In [5]:
sql = '''
SELECT *
FROM dividend
WHERE name = '%s'
'''
sql = sql % name
dividend = pd.read_sql(sql, const)
dividend.drop(['PRICE', 'PERCENT'], axis=1, inplace=True)
dividend.columns = dividend.columns.str.lower()
dividend['shares'] = dividend['shares'].astype('int64')
dividend['xdate'] = pd.to_datetime(dividend['xdate'])
dividend['paiddate'] = pd.to_datetime(dividend['paiddate'])
dividend.style.format(format_dict)

,name,q4,q3,q2,q1,dividend,shares,xdate,paiddate,kind,actual
0,PTTEP,3.0000,0.0000,4.2500,0.0000,7.2500,600,2022-08-15,2022-08-26,,1


In [6]:
sql = """
SELECT * FROM stocks WHERE name = '%s'
"""
sql = sql % name
print(sql)

stocks = pd.read_sql(sql, conlite)
stocks.style.format(format_dict)


SELECT * FROM stocks WHERE name = 'PTTEP'



,id,name,max_price,min_price,status,buy_target,sell_target,volume,beta,cost,qty,buy_spread,sell_spread,available_qty,bl,sh,reason,market
0,9718,PTTEP,0.00,0.00,B,146.00,160.00,0.00,0.00,155.00,600,-4,4,600,0,0,RD05pct,SET50


### Create orders from stocks after adjust stocks in port_lite from consensus

In [7]:
sql = '''
DELETE FROM orders'''
rp = conlite.execute(sql)
rp.rowcount

18

In [8]:
sql = '''
SELECT status AS trade, name, qty, buy_target AS price, 0 AS active, reason, market
FROM stocks
WHERE status IN ('B','O')
ORDER BY name'''
df_buy = pd.read_sql(sql, conlite)
df_buy

,trade,name,qty,price,active,reason,market
0,O,AP,10000,9.50,0,MinP,SET100
1,O,ASK,3000,30.00,0,52WL,SET
2,O,BANPU,12000,11.50,0,1L,SET50
3,O,BCPG,9000,10.00,0,52WL,SET100
4,O,DELTA,300,450.00,0,52WL,SET
5,O,IVL,3000,43.50,0,13WL,SET50
6,B,MCS,5000,10.60,0,52WL,SET
7,B,PTTEP,600,146.00,0,RD05pct,SET50
8,O,RBF,9000,11.40,0,52WL,SET100
9,O,SINGER,3000,35.50,0,52WL,SET100


In [9]:
sql = '''
SELECT status AS trade, name, qty, sell_target AS price, 0 AS active, reason, market
FROM stocks
WHERE status IN ('I','S')'''
df_sell = pd.read_sql(sql, conlite)
df_sell

,trade,name,qty,price,active,reason,market
0,I,DIF,10000,14.6,0,CUT,SET
1,I,SCC,300,400.0,0,C398,SET50
2,I,BCH,3000,21.0,0,CP1S,SET100
3,I,DOHOME,2800,20.3,0,RD20pct,SET100
4,I,RCL,3000,46.0,0,RD20pct,SET100


In [10]:
df_order = df_buy.append(df_sell, ignore_index=True)
df_order.shape

(18, 7)

In [11]:
df_order.groupby(['trade']).count()

,name,qty,price,active,reason,market
trade,,,,,,
B,3,3,3,3,3,3
I,5,5,5,5,5,5
O,10,10,10,10,10,10


In [12]:
df_order.loc[df_order.trade == 'I','trade'] = 'S'
df_order.groupby(['trade']).count()

,name,qty,price,active,reason,market
trade,,,,,,
B,3,3,3,3,3,3
O,10,10,10,10,10,10
S,5,5,5,5,5,5


In [13]:
sql = '''
SELECT name, xdate
FROM dividend
'''
dividend = pd.read_sql(sql, const)
dividend.columns = dividend.columns.str.lower()
dividend['xdate'] = pd.to_datetime(dividend['xdate'])
dividend.shape

(83, 2)

In [14]:
df_merge = pd.merge(df_order, dividend, on='name', how='outer', indicator=True)
df_merge.shape

(89, 9)

### If there is no dividend record, must create dividend record before proceed

In [15]:
no_div = df_merge[df_merge['_merge'] == 'left_only']
no_div

,trade,name,qty,price,active,reason,market,xdate,_merge
3,O,BCPG,9000.0,10.00,0.0,52WL,SET100,NaT,left_only
4,O,DELTA,300.0,450.00,0.0,52WL,SET,NaT,left_only
8,O,RBF,9000.0,11.40,0.0,52WL,SET100,NaT,left_only
9,O,SINGER,3000.0,35.50,0.0,52WL,SET100,NaT,left_only
10,O,SSP,10000.0,8.75,0.0,52WL,SET,NaT,left_only
12,O,TTB,90000.0,0.95,0.0,52WL,SET50,NaT,left_only


### Rerun from here

In [16]:
# Include both having diviedend and having no dividend records
mask = ((df_merge['_merge'] == 'both') | (df_merge['_merge'] == 'left_only'))
df_orders = df_merge[mask].copy()
df_orders.drop(['_merge'], axis=1, inplace=True)
df_orders['qty'] = df_orders['qty'].astype('int64')
df_orders['active'] = df_orders['active'].astype('int64')
df_orders['xdate'] = df_orders['xdate'].astype('str')
df_orders.groupby('trade').count()

,name,qty,price,active,reason,market,xdate
trade,,,,,,,
B,3,3,3,3,3,3,3
O,10,10,10,10,10,10,10
S,5,5,5,5,5,5,5


### Set xdate 2 weeks window of first Tuesday and second Friday

In [17]:
fm_date = '2022-05-03'

### Restart and Run All Cells

In [18]:
fltr_sts_o = (df_orders.trade == 'O') 
df_sts_o = df_orders[fltr_sts_o]
df_sts_o

,trade,name,qty,price,active,reason,market,xdate
0,O,AP,10000,9.50,0,MinP,SET100,2022-05-10
1,O,ASK,3000,30.00,0,52WL,SET,2022-03-03
2,O,BANPU,12000,11.50,0,1L,SET50,2022-04-11
3,O,BCPG,9000,10.00,0,52WL,SET100,NaT
4,O,DELTA,300,450.00,0,52WL,SET,NaT
5,O,IVL,3000,43.50,0,13WL,SET50,2022-05-31
8,O,RBF,9000,11.40,0,52WL,SET100,NaT
9,O,SINGER,3000,35.50,0,52WL,SET100,NaT
10,O,SSP,10000,8.75,0,52WL,SET,NaT
12,O,TTB,90000,0.95,0,52WL,SET50,NaT


In [19]:
fltr_sts_b = (df_orders.trade == 'B') 
df_sts_b = df_orders[fltr_sts_b]
df_sts_b

,trade,name,qty,price,active,reason,market,xdate
6,B,MCS,5000,10.6,0,52WL,SET,2022-09-03
7,B,PTTEP,600,146.0,0,RD05pct,SET50,2022-08-15
11,B,TMT,12000,7.9,0,RD20pct,SET,2022-08-27


In [20]:
fltr_sts_s = (df_orders.trade == 'S')
df_sts_s = df_orders[fltr_sts_s]
df_sts_s.sort_values('name',ascending=True)

,trade,name,qty,price,active,reason,market,xdate
15,S,BCH,3000,21.0,0,CP1S,SET100,2022-08-31
13,S,DIF,10000,14.6,0,CUT,SET,2022-08-16
16,S,DOHOME,2800,20.3,0,RD20pct,SET100,2022-02-02
17,S,RCL,3000,46.0,0,RD20pct,SET100,2022-08-27
14,S,SCC,300,400.0,0,C398,SET50,2022-08-10


In [21]:
frames = [df_sts_b, df_sts_o, df_sts_s]
df_orders = pd.concat(frames, sort=False)
df_orders.sort_values(['trade','name'],ascending=True)

,trade,name,qty,price,active,reason,market,xdate
6,B,MCS,5000,10.60,0,52WL,SET,2022-09-03
7,B,PTTEP,600,146.00,0,RD05pct,SET50,2022-08-15
11,B,TMT,12000,7.90,0,RD20pct,SET,2022-08-27
0,O,AP,10000,9.50,0,MinP,SET100,2022-05-10
1,O,ASK,3000,30.00,0,52WL,SET,2022-03-03
2,O,BANPU,12000,11.50,0,1L,SET50,2022-04-11
3,O,BCPG,9000,10.00,0,52WL,SET100,NaT
4,O,DELTA,300,450.00,0,52WL,SET,NaT
5,O,IVL,3000,43.50,0,13WL,SET50,2022-05-31
8,O,RBF,9000,11.40,0,52WL,SET100,NaT


In [22]:
df_orders.loc[df_orders.trade == 'O','trade'] = 'B'
df_orders.groupby(['trade']).count()

,name,qty,price,active,reason,market,xdate
trade,,,,,,,
B,13,13,13,13,13,13,13
S,5,5,5,5,5,5,5


In [23]:
rcds = df_orders.values.tolist()
len(rcds)

18

In [24]:
for rcd in rcds:
    print(rcd)

['B', 'MCS', 5000, 10.6, 0, '52WL', 'SET', '2022-09-03']
['B', 'PTTEP', 600, 146.0, 0, 'RD05pct', 'SET50', '2022-08-15']
['B', 'TMT', 12000, 7.9, 0, 'RD20pct', 'SET', '2022-08-27']
['B', 'AP', 10000, 9.5, 0, 'MinP', 'SET100', '2022-05-10']
['B', 'ASK', 3000, 30.0, 0, '52WL', 'SET', '2022-03-03']
['B', 'BANPU', 12000, 11.5, 0, '1L', 'SET50', '2022-04-11']
['B', 'BCPG', 9000, 10.0, 0, '52WL', 'SET100', 'NaT']
['B', 'DELTA', 300, 450.0, 0, '52WL', 'SET', 'NaT']
['B', 'IVL', 3000, 43.5, 0, '13WL', 'SET50', '2022-05-31']
['B', 'RBF', 9000, 11.4, 0, '52WL', 'SET100', 'NaT']
['B', 'SINGER', 3000, 35.5, 0, '52WL', 'SET100', 'NaT']
['B', 'SSP', 10000, 8.75, 0, '52WL', 'SET', 'NaT']
['B', 'TTB', 90000, 0.95, 0, '52WL', 'SET50', 'NaT']
['S', 'DIF', 10000, 14.6, 0, 'CUT', 'SET', '2022-08-16']
['S', 'SCC', 300, 400.0, 0, 'C398', 'SET50', '2022-08-10']
['S', 'BCH', 3000, 21.0, 0, 'CP1S', 'SET100', '2022-08-31']
['S', 'DOHOME', 2800, 20.3, 0, 'RD20pct', 'SET100', '2022-02-02']
['S', 'RCL', 3000, 46.0

In [25]:
sql = """
INSERT INTO orders (trade, name, qty, price, active,
reason, market, xdate)
VALUES (?, ?, ?, ?, ?, ?, ?, ?)"""
print(sql)


INSERT INTO orders (trade, name, qty, price, active,
reason, market, xdate)
VALUES (?, ?, ?, ?, ?, ?, ?, ?)


In [26]:
for rcd in rcds:
    conlite.execute(sql, rcd)

In [27]:
sql = """
SELECT * FROM orders"""
df_orders = pd.read_sql(sql, conlite)
df_orders

,id,trade,name,qty,price,active,reason,market,xdate
0,1,B,MCS,5000,10.60,0,52WL,SET,2022-09-03
1,2,B,PTTEP,600,146.00,0,RD05pct,SET50,2022-08-15
2,3,B,TMT,12000,7.90,0,RD20pct,SET,2022-08-27
3,4,B,AP,10000,9.50,0,MinP,SET100,2022-05-10
4,5,B,ASK,3000,30.00,0,52WL,SET,2022-03-03
5,6,B,BANPU,12000,11.50,0,1L,SET50,2022-04-11
6,7,B,BCPG,9000,10.00,0,52WL,SET100,NaT
7,8,B,DELTA,300,450.00,0,52WL,SET,NaT
8,9,B,IVL,3000,43.50,0,13WL,SET50,2022-05-31
9,10,B,RBF,9000,11.40,0,52WL,SET100,NaT


In [28]:
df_orders.loc[(df_orders.xdate == 'NaT'),['xdate']] = ['2022-02-02']
df_orders

,id,trade,name,qty,price,active,reason,market,xdate
0,1,B,MCS,5000,10.60,0,52WL,SET,2022-09-03
1,2,B,PTTEP,600,146.00,0,RD05pct,SET50,2022-08-15
2,3,B,TMT,12000,7.90,0,RD20pct,SET,2022-08-27
3,4,B,AP,10000,9.50,0,MinP,SET100,2022-05-10
4,5,B,ASK,3000,30.00,0,52WL,SET,2022-03-03
5,6,B,BANPU,12000,11.50,0,1L,SET50,2022-04-11
6,7,B,BCPG,9000,10.00,0,52WL,SET100,2022-02-02
7,8,B,DELTA,300,450.00,0,52WL,SET,2022-02-02
8,9,B,IVL,3000,43.50,0,13WL,SET50,2022-05-31
9,10,B,RBF,9000,11.40,0,52WL,SET100,2022-02-02


In [29]:
pd.read_sql_query('SELECT * FROM orders', conlite)

,id,trade,name,qty,price,active,reason,market,xdate
0,1,B,MCS,5000,10.60,0,52WL,SET,2022-09-03
1,2,B,PTTEP,600,146.00,0,RD05pct,SET50,2022-08-15
2,3,B,TMT,12000,7.90,0,RD20pct,SET,2022-08-27
3,4,B,AP,10000,9.50,0,MinP,SET100,2022-05-10
4,5,B,ASK,3000,30.00,0,52WL,SET,2022-03-03
5,6,B,BANPU,12000,11.50,0,1L,SET50,2022-04-11
6,7,B,BCPG,9000,10.00,0,52WL,SET100,NaT
7,8,B,DELTA,300,450.00,0,52WL,SET,NaT
8,9,B,IVL,3000,43.50,0,13WL,SET50,2022-05-31
9,10,B,RBF,9000,11.40,0,52WL,SET100,NaT


In [30]:
sqlUpd = """
UPDATE orders
SET xdate = '2022-02-02'
WHERE xdate = 'NaT'"""
rp = conlite.execute(sqlUpd)
rp.rowcount

6

In [31]:
pd.read_sql_query('SELECT * FROM orders WHERE qty = 0', conlite)

,id,trade,name,qty,price,active,reason,market,xdate


In [32]:
file_name = 'orders.csv'
data_file = data_path + file_name
output_file = csv_path + file_name
box_file = box_path + file_name
one_file = one_path + file_name

df_orders[cols].sort_values(['trade','name'],ascending=[True,True]).to_csv(output_file, header=True, index=False)
df_orders[cols].sort_values(['trade','name'],ascending=[True,True]).to_csv(data_file, header=True, index=False)
df_orders[cols].sort_values(['trade','name'],ascending=[True,True]).to_csv(box_file, header=True, index=False)
df_orders[cols].sort_values(['trade','name'],ascending=[True,True]).to_csv(one_file, header=True, index=False)

### Process to set target price

### 1) Set50 records

In [33]:
pd.read_sql_query('SELECT * FROM orders WHERE market = "SET50" ORDER BY trade, name', conlite)

,id,trade,name,qty,price,active,reason,market,xdate
0,6,B,BANPU,12000,11.50,0,1L,SET50,2022-04-11
1,9,B,IVL,3000,43.50,0,13WL,SET50,2022-05-31
2,2,B,PTTEP,600,146.00,0,RD05pct,SET50,2022-08-15
3,13,B,TTB,90000,0.95,0,52WL,SET50,2022-02-02
4,15,S,SCC,300,400.00,0,C398,SET50,2022-08-10


In [34]:
name = 'TOP'
limit = 60 # 1 month of data
sql = """
SELECT *
FROM price
WHERE name = '%s'
ORDER BY date DESC
LIMIT %s"""
sql = sql % (name, limit)
df = pd.read_sql(sql, const)
df.describe()

,price,maxp,minp,qty,opnp
count,60.000000,60.000000,60.000000,6.000000e+01,60.000000
mean,53.700000,54.350000,52.995833,1.643531e+07,53.729167
std,3.473202,3.538589,3.441857,1.219397e+07,3.468775
min,49.250000,49.750000,48.250000,3.331129e+06,49.750000
25%,51.000000,51.250000,50.000000,9.087441e+06,50.500000
50%,52.125000,52.500000,51.375000,1.289439e+07,52.000000
75%,57.000000,57.750000,56.562500,1.956076e+07,57.062500
max,60.750000,62.250000,60.500000,6.425642e+07,61.000000


### 2) Set100 records

In [35]:
pd.read_sql_query('SELECT * FROM orders WHERE market = "SET100" ORDER BY trade, name', conlite)

,id,trade,name,qty,price,active,reason,market,xdate
0,4,B,AP,10000,9.5,0,MinP,SET100,2022-05-10
1,7,B,BCPG,9000,10.0,0,52WL,SET100,2022-02-02
2,10,B,RBF,9000,11.4,0,52WL,SET100,2022-02-02
3,11,B,SINGER,3000,35.5,0,52WL,SET100,2022-02-02
4,16,S,BCH,3000,21.0,0,CP1S,SET100,2022-08-31
5,17,S,DOHOME,2800,20.3,0,RD20pct,SET100,2022-02-02
6,18,S,RCL,3000,46.0,0,RD20pct,SET100,2022-08-27


In [36]:
name = 'KKP'
limit = 60 # 1 month of data
sql = """
SELECT *
FROM price
WHERE name = '%s'
ORDER BY date DESC
LIMIT %s"""
sql = sql % (name, limit)
df = pd.read_sql(sql, const)
df.describe()

,price,maxp,minp,qty,opnp
count,60.000000,60.000000,60.000000,6.000000e+01,60.000000
mean,66.158333,66.883333,65.541667,5.663264e+06,66.275000
std,3.777433,3.704803,3.695269,3.543826e+06,3.697715
min,60.500000,61.500000,59.500000,1.763119e+06,60.750000
25%,62.750000,63.500000,62.250000,3.419184e+06,63.000000
50%,65.125000,66.000000,64.750000,4.667822e+06,65.125000
75%,70.250000,71.000000,69.562500,6.947426e+06,70.250000
max,72.000000,72.250000,71.250000,2.203634e+07,72.000000


### 3) Set records

In [37]:
pd.read_sql_query('SELECT * FROM orders WHERE market = "SET" ORDER BY trade, name', conlite)

,id,trade,name,qty,price,active,reason,market,xdate
0,5,B,ASK,3000,30.00,0,52WL,SET,2022-03-03
1,8,B,DELTA,300,450.00,0,52WL,SET,2022-02-02
2,1,B,MCS,5000,10.60,0,52WL,SET,2022-09-03
3,12,B,SSP,10000,8.75,0,52WL,SET,2022-02-02
4,3,B,TMT,12000,7.90,0,RD20pct,SET,2022-08-27
5,14,S,DIF,10000,14.60,0,CUT,SET,2022-08-16


In [38]:
name = 'AIT'
limit = 120 # 1 month of data
sql = """
SELECT *
FROM price
WHERE name = '%s'
ORDER BY date DESC
LIMIT %s"""
sql = sql % (name, limit)
df = pd.read_sql(sql, const)
df.describe()

,price,maxp,minp,qty,opnp
count,120.000000,120.000000,120.000000,1.200000e+02,120.000000
mean,6.562083,6.704167,6.461250,1.255295e+07,6.572500
std,0.618245,0.653902,0.594423,1.986173e+07,0.615311
min,5.550000,5.650000,5.500000,8.111610e+05,5.550000
25%,5.987500,6.050000,5.900000,2.882052e+06,6.000000
50%,6.700000,6.850000,6.600000,6.191094e+06,6.700000
75%,7.000000,7.150000,6.900000,1.344754e+07,7.000000
max,8.050000,8.350000,7.900000,1.238636e+08,8.050000


### After call Ord-log.bat

In [39]:
def categorise(row):  
    if row['trade'] == 'B':
        return 'Buy'
    else:
        return 'Sell'

In [40]:
df = pd.read_csv('../data/orders-log.csv')
df['trans'] = df.apply(lambda row: categorise(row), axis=1)

df_tab = pn.widgets.Tabulator(df[colt], layout='fit_data', width=800)
df_tab

Tabulator(layout='fit_data', value=   trans    name  ..., width=800)

In [41]:
df[colt]

,trans,name,spd,reason,qty,target,current,percent,active,xdate
0,Buy,AP,-14,MinP,10000,9.50,10.40,+1.96%,0,2022-05-10
1,Buy,ASK,-31,52WL,3000,30.00,37.75,+3.42%,0,2022-03-03
2,Buy,BANPU,-13,1L,12000,11.50,12.80,0.00%,0,2022-04-11
3,Buy,BCPG,-3,52WL,9000,10.00,10.30,-0.96%,0,2022-02-02
4,Buy,DELTA,-45,52WL,300,450.00,540.00,-0.37%,0,2022-02-02
5,Buy,IVL,2,13WL,3000,43.50,43.00,-2.27%,0,2022-05-31
6,Buy,MCS,-7,52WL,5000,10.60,11.30,0.00%,0,2022-09-03
7,Buy,PTTEP,-10,RD05pct,600,146.00,151.00,-2.27%,0,2022-08-15
8,Buy,RBF,-18,52WL,9000,11.40,13.20,0.00%,0,2022-02-02
9,Buy,SINGER,-42,52WL,3000,35.50,46.00,+2.79%,0,2022-02-02
